<a href="https://colab.research.google.com/github/michaellaic/NLP3/blob/main/RNN_and_LTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sumy
!pip install spacy
!pip install tensorflow keras nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 19.8 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=c3054f28acb85af916fc03fb671502e29e63e83f1223c476b4753889ba00ae0c
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=9aff7d3fde4de9a526bac5f15838984dc4c54a41c184d4181f367dac2e484137
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [2]:


import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.utils.extmath import randomized_svd
import spacy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, Dense
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.kl import KLSummarizer
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import nltk

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as SumyTokenizer
from sumy.summarizers.kl import KLSummarizer
import nltk
import re

nltk.download('punkt')
nltk.download('wordnet')
!python -m nltk.downloader stopwords



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Create Corpus

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Neuro-linguistic_programming'

response = requests.get(url)
soup_text = ''
if response.status_code == 200:
    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract all text from paragraph tags
    paragraphs = soup.find_all('p')
    for para in paragraphs:
        soup_text += para.get_text()
        #print(para.get_text())
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

corpus = soup_text

In [4]:
corpus = corpus.replace('\n', ' ')
corpus = corpus.lower()

pattern = r'\[\d+\]'
corpus = re.sub(pattern, '', corpus)


In [5]:
pattern = r'[.,]'
corpus_segments = re.split(pattern, corpus)
corpus_segments = [segment.strip() for segment in corpus_segments]

# Create input for network

In [6]:
text_data = corpus_segments

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [7]:
# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# RNN

## Define model

In [8]:
rnn_model = Sequential()
rnn_model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
rnn_model.add(SimpleRNN(128))  # Replace LSTM with SimpleRNN
rnn_model.add(Dense(total_words, activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

#rnn_model.fit(X, y, epochs=50, verbose=1)
rnn_model.fit(X, y, epochs=3, verbose=1)

Epoch 1/3
103/103 [==============================] - 10s 71ms/step - loss: 6.6756 - accuracy: 0.0328
Epoch 2/3
103/103 [==============================] - 5s 53ms/step - loss: 6.2072 - accuracy: 0.0377
Epoch 3/3
103/103 [==============================] - 8s 73ms/step - loss: 6.1454 - accuracy: 0.0410


## Seed next words

In [9]:
next_words = 5

seed_text = "i love nlp"
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = rnn_model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 31ms/step
Next predicted words: i love nlp of of of of of


# LSTM

## Define model

In [10]:
lstm_model = Sequential()
lstm_model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(total_words, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


#lstm_model.fit(X, y, epochs=250, verbose=1)
lstm_model.fit(X, y, epochs=3, verbose=1)

Epoch 1/3
103/103 [==============================] - 13s 102ms/step - loss: 6.6457 - accuracy: 0.0377
Epoch 2/3
103/103 [==============================] - 12s 114ms/step - loss: 6.2044 - accuracy: 0.0398
Epoch 3/3
103/103 [==============================] - 12s 114ms/step - loss: 6.1257 - accuracy: 0.0401


## Seed next words

In [11]:
next_words = 5

seed_text = "i love nlp"
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = lstm_model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 35ms/step
Next predicted words: i love nlp the the the the the


# Comapring

In [12]:
def calculate_perplexity(model, X_val, y_val):
    total_log_prob = 0
    total_words = 0
    for seq, next_word_array in zip(X_val, y_val):

        seq_padded = pad_sequences([seq], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(seq_padded, verbose=0)[0]

        # Assuming next_word_array contains the index of the next word
        next_word = int(next_word_array[0])  # Extracting the integer index

        prob = predicted_probs[next_word]
        total_log_prob += np.log(prob)
        total_words += 1

    avg_log_prob = total_log_prob / total_words
    perplexity = np.exp(-avg_log_prob)
    return perplexity

In [13]:
#perplexity_rnn = calculate_perplexity(rnn_model, X, y)
#perplexity_lstm = calculate_perplexity(lstm_model, X, y)

# Summarization on the corpus - KL-Sum algorithm

In [14]:
parser = PlaintextParser.from_string(corpus, SumyTokenizer("english"))

# Using KL Summarizer
summarizer = KLSummarizer()

# Summarize the document with 4 sentences
summary = summarizer(parser.document, 4)

# Print the summary
for sentence in summary:
    print(sentence)

their 1975 book, the structure of magic i: a book about language and therapy, is intended to be a codification of the therapeutic techniques of perls and satir.
there is no sense in which bandler and grinder caused or participated in a paradigm shift.
the practitioner thereafter assists the client in achieving the desired outcomes by using certain tools and techniques to change internal representations and responses to stimuli in the world.
bandler and grinder were influenced by the shamanism described in the books of carlos castaneda.


In [15]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

# Tokenize the corpus into sentences
sentences = nltk.sent_tokenize(corpus)

# Initialize counters
positive, negative, neutral = 0, 0, 0

for sentence in sentences:
    score = sia.polarity_scores(sentence)
    if score['compound'] >= 0.05:
        positive += 1
    elif score['compound'] <= -0.05:
        negative += 1
    else:
        neutral += 1

total = positive + negative + neutral

# Calculate percentages
positive_pct = (positive / total) * 100
negative_pct = (negative / total) * 100
neutral_pct = (neutral / total) * 100

# Print the results
print(f"Total sentences: {total}")
print(f"Positive: {positive} ({positive_pct:.2f}%)")
print(f"Negative: {negative} ({negative_pct:.2f}%)")
print(f"Neutral: {neutral} ({neutral_pct:.2f}%)")

Total sentences: 143
Positive: 45 (31.47%)
Negative: 39 (27.27%)
Neutral: 59 (41.26%)


##Fine-Tune GPT-2 Model

In [17]:
#!pip install transformers datasets torch
import torch
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

In [19]:
# Your provided corpus (assuming it's stored in a variable named `corpus`)
# Tokenize the corpus into sentences
sentences = nltk.sent_tokenize(corpus)


# Load pretrained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Prepare training data
inputs = tokenizer(corpus, return_tensors='tf', max_length=512, truncation=True, padding="max_length")
dataset = tf.data.Dataset.from_tensor_slices((inputs['input_ids'], inputs['attention_mask']))
dataset = dataset.shuffle(1000).batch(2)

# Define a simple training loop
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

@tf.function
def train_step(input_ids, attention_mask):
    with tf.GradientTape() as tape:
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training
epochs = 3
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    for batch in dataset:
        input_ids, attention_mask = batch
        loss = train_step(input_ids, attention_mask)
        print(f'Loss: {loss.numpy()}')

# 5 sentences from the corpus
partial_sentences = [' '.join(sentence.split()[:len(sentence.split()) // 2]) for sentence in sentences[:5]]


for i, (full_sentence, prompt) in enumerate(zip(sentences[:5], partial_sentences), 1):
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    output_sequences = model.generate(input_ids, max_length=50, num_return_sequences=1)
    completion = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    print(f"Example {i}:\nActual sentence:\n{full_sentence}\nPartial sentence:\n{prompt}\nCompleted by model:\n{completion.split('.')[0]}.\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Example 2:
Actual sentence:
according to bandler and grinder, nlp can treat problems such as phobias, depression, tic disorders, psychosomatic illnesses, near-sightedness,[a] allergy, the common cold,[a] and learning disorders, often in a single session.
Partial sentence:
according to bandler and grinder, nlp can treat problems such as phobias, depression, tic disorders,
Completed by model:
according to bandler and grinder, nlp can treat problems such as phobias, depression, tic disorders, and other mental health problems.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example 3:
Actual sentence:
they also say that nlp can model the skills of exceptional people, allowing anyone to acquire them.
Partial sentence:
they also say that nlp can model the
Completed by model:
they also say that nlp can model the behavior of the brain in a way that is similar to that of the brain in a mouse.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Example 4:
Actual sentence:
[b] nlp has been adopted by some hypnotherapists as well as by companies that run seminars marketed as leadership training to businesses and government agencies.
Partial sentence:
[b] nlp has been adopted by some hypnotherapists as well as by
Completed by model:
[b] nlp has been adopted by some hypnotherapists as well as by others.

Example 5:
Actual sentence:
there is no scientific evidence supporting the claims made by nlp advocates, and it has been called a pseudoscience.
Partial sentence:
there is no scientific evidence supporting the claims made
Completed by model:
there is no scientific evidence supporting the claims made by the authors of this paper," said Dr.



##Sentiment Analysis

In [21]:
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentences = nltk.sent_tokenize(corpus)
sentiments = [analyzer.polarity_scores(sentence) for sentence in sentences]

# Aggregate and calculate the percentage of each sentiment
positive = sum(1 for s in sentiments if s['compound'] > 0.05) / len(sentiments)
negative = sum(1 for s in sentiments if s['compound'] < -0.05) / len(sentiments)
neutral = sum(1 for s in sentiments if -0.05 <= s['compound'] <= 0.05) / len(sentiments)

print(f"Positive: {positive*100:.2f}%")
print(f"Negative: {negative*100:.2f}%")
print(f"Neutral: {neutral*100:.2f}%")

Positive: 32.87%
Negative: 26.57%
Neutral: 40.56%
